In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!git clone https://github.com/saurabhhumane125/newssumm-summarization.git


Cloning into 'newssumm-summarization'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5 (delta 0), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), done.


In [ ]:
%cd newssumm-summarization


/content/newssumm-summarization


In [ ]:
!ls


notebooks


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/newssumm/multidoc_v1.csv")
print("Rows:", len(df))
df.head(1)


Rows: 306


,input_text,target_summary,num_docs,category,date
0,[DOC_1]\nNew Delhi: Bajaj Auto and TVS Motor C...,TVS Motor Company and Bajaj Auto have agreed t...,3,Automotive,2019-10-31


In [ ]:
!pip install -q transformers datasets evaluate rouge-score bert-score accelerate


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import transformers
print(transformers.__version__)

from transformers import BartTokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
print("Tokenizer loaded successfully")


4.57.3


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Tokenizer loaded successfully


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/newssumm/multidoc_v1.csv")
print("Rows:", len(df))
df.head(1)


Rows: 306


,input_text,target_summary,num_docs,category,date
0,[DOC_1]\nNew Delhi: Bajaj Auto and TVS Motor C...,TVS Motor Company and Bajaj Auto have agreed t...,3,Automotive,2019-10-31


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset


Dataset({
    features: ['input_text', 'target_summary', 'num_docs', 'category', 'date'],
    num_rows: 306
})

In [ ]:
def preprocess(batch):
    model_inputs = tokenizer(
        batch["input_text"],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["target_summary"],
            max_length=256,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset.column_names
)


Map:   0%|          | 0/306 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(
    test_size=0.1,
    seed=42
)

tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 275
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 31
    })
})

In [ ]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bart_multidoc",
    eval_strategy="epoch",          # <-- FIXED
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=20,
    save_total_limit=2,
    report_to="none"
)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)


/tmp/ipython-input-2542952513.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,3.539600,2.564404
2,2.377500,1.867289
3,1.988500,1.735120


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=207, training_loss=3.0623606161218913, metrics={'train_runtime': 138.6757, 'train_samples_per_second': 5.949, 'train_steps_per_second': 1.493, 'total_flos': 503032578048000.0, 'train_loss': 3.0623606161218913, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/newssumm/bart_multidoc_v1")


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained(
    "/content/drive/MyDrive/newssumm/bart_multidoc_v1"
)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


In [ ]:
sample = df.iloc[0]["input_text"]

inputs = tokenizer(
    sample,
    return_tensors="pt",
    truncation=True,
    max_length=1024
).to(model.device)

summary_ids = model.generate(
    **inputs,
    max_length=200,
    num_beams=4,
    early_stopping=True
)

generated_summary = tokenizer.decode(
    summary_ids[0],
    skip_special_tokens=True
)

print("GENERATED SUMMARY:\n", generated_summary)
print("\nREFERENCE SUMMARY:\n", df.iloc[0]["target_summary"])


GENERATED SUMMARY:
 Bajaj Auto and TVS Motor Company have entered into a settlement agreement to amicably settle a decade-old dispute related to alleged infringement of Bajaj's patent for digital twin spark plug ignition technology. The settlement includes defamation suit against Bajaja Auto filed by TVS Motors and other related proceedings filed against each other, which were pending before various courts in Sri Lanka and Mexico. The parties have agreed to withdraw the pending proceedings and release each other from all liabilities, claims, demands and actions in respect of the pending proceeding.

REFERENCE SUMMARY:
 TVS Motor Company and Bajaj Auto have agreed to settle a decade-old legal dispute over twin spark plug engine technologies. The dispute arose from alleged infringement of Bajaj's patent No 195904, a defamation suit filed by TVS against Bajaj Auto, and related proceedings pending before various courts. The agreement allows TVS and Bajaj to withdraw pending proceedings and

In [ ]:
import evaluate
import torch

rouge = evaluate.load("rouge")

def generate_summary(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(model.device)

    summary_ids = model.generate(
        **inputs,
        max_length=200,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

preds = []
refs = []

for i in range(len(df)):
    preds.append(generate_summary(df.iloc[i]["input_text"]))
    refs.append(df.iloc[i]["target_summary"])

results = rouge.compute(predictions=preds, references=refs)
results


KeyboardInterrupt: 

In [ ]:
len(preds), len(refs)


(211, 211)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=preds, references=refs)
results


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


NameError: name 'preds' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/newssumm/multidoc_v1.csv")

from transformers import BartForConditionalGeneration, BartTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = BartForConditionalGeneration.from_pretrained(
    "/content/drive/MyDrive/newssumm/bart_multidoc_v1"
).to(device)

tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
  df_eval = df.sample(n=100, random_state=42).reset_index(drop=True)


In [ ]:
from tqdm import tqdm

def batch_generate(texts, batch_size=2):
    summaries = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(device)

        with torch.no_grad():
            summary_ids = model.generate(
                **inputs,
                max_length=200,
                num_beams=4,
                early_stopping=True
            )

        summaries.extend(
            tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
        )

    return summaries


In [ ]:
preds = batch_generate(df_eval["input_text"].tolist())
refs = df_eval["target_summary"].tolist()

len(preds), len(refs)


100%|██████████| 50/50 [03:47<00:00,  4.56s/it]


(100, 100)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=preds, references=refs)
results


{'rouge1': np.float64(0.3288199228754886),
 'rouge2': np.float64(0.14337708880199923),
 'rougeL': np.float64(0.22452788293910125),
 'rougeLsum': np.float64(0.232698102993039)}